# PepperOpenDayDemo

## Environment Setup

In [1]:
# built-in imports
# !pip install requests
# !pip install speechrecognition==3.8.1
# !pip install paramiko
# !pip install pandas
# !pip install beautifulsoup4
# !pip install pocketsphinx
# !pip install scipy

In [3]:
from traceback import format_exc
from time      import sleep

from idle import idling

from interactiveControls import EyesController
from interactiveControls import track_head
from interactiveControls import return_to_default_pos

from humanInteraction import detect
from humanInteraction import listen
from humanInteraction import speak
from humanInteraction import faceTracker
from humanInteraction import classifyQuery
from humanInteraction import stopListening

from chatflow import convFlow

from behaviorManager import launchAndStopBehavior, service

RuntimeError: System error: Connection refused

## Constants

In [ ]:
### Networking Constants ###
GPT_HOST = "10.104.23.208"
PEPPER_HOST = "10.104.23.185"
GPT_PORT = 8891
PEPPER_PORT = 9559

### Audio Processing Constants ###
PEPPER_RECORDINGS_PATH = "/home/nao/microphones/recording.wav"
SERVER_RECORDINGS_PATH = "recordings/recording.wav"

### Display Generation Constants ###
FILE_NAME_TEMP = "display.html"
PEPPER_HTML_PATH = "/home/nao/.local/share/PackageManager/apps/robot-page/html/"  # page.html"
PEPPER_PAGE_LANDING = PEPPER_HTML_PATH + "page.html" # '/home/nao/.local/share/PackageManager/apps/robot-page/html/page.html'
PEPPER_QR_LANDING = PEPPER_HTML_PATH + "webfiles/qr.png"  # switched slash direction
PEPPER_IMG_LANDING = PEPPER_HTML_PATH + "webfiles/img.png"  # switched slash direction
TEXT_BY_ID_PATH = "textbyID.csv"

# Choregraph Behavior

In [ ]:
launchAndStopBehavior(service, '.lastUploadedChoregrapheBehavior/behavior_1')

## MAIN

In [ ]:
global idle
global responsesPipeline
global eyes

idle = idling()
eyes = EyesController()
eyes.startEyes("neutral")

try:
    track_head()

    while True:
        # Empty sentences list
        responsesPipeline = []
        
        # Idle behaviors to keep peppers temperature low
        idle.start_idle_behavior()

        # trigger smart listen
        detect(idle)
        faceTracker()
        
        rCount = 0
        cCounter = 0
        while rCount < 3:
            query = listen()
            if query == '%low_volume_error%':
                speak("Sorry, I didn't quite catch that. Could you please repeat that?")
                rCount += 1
            else:
                pred_class, thresh = classifyQuery(query)

                if not thresh:
                    rCount += 1
                if pred_class == 'Courses':
                    cCounter += 1
                if cCounter == 2:
                    pred_class = 'SpecificCourse'

                repeat = convFlow(query, pred_class, thresh, responsesPipeline, eyes, rCount)

                if repeat:
                    rCount += 1
        
        # When the user has tried 3 times, return to default position
        sleep(10)
        return_to_default_pos()
        continue
except Exception as e:
    print(e)
    print(format_exc())
finally:
    stopListening()
    return_to_default_pos()
    idle.start_idle_behavior()
    print('Goodbye')